In [2]:
import pickle
from collections import Counter
from nltk.corpus import stopwords
import nltk



# nltk.download('stopwords')

eng_stopwords = set(stopwords.words('english'))

In [4]:
pickle_files = {'abstract_sentences': "AbstractSentences2007.pkl", 
                'abstracts': "Abstracts2007.pkl",
               'full_text_sentences': "FullTextSentences2007.pkl", 
                'full_text': "FullTexts2007.pkl",
                'trained_vectors': "trainedVectors2007.pkl"}

p_abstract_sentences = open("FullTextSentences2007.pkl","rb")

for varname, p in pickle_files.items():
    pickled_opened_file = open(p, "rb")
    exec(varname + " = pickle.load(pickled_opened_file)")
    

In [17]:
def keep_word(word):
    return word not in eng_stopwords and word != ',' and word != '.' and word != '\n'

In [18]:
word_counts_map = []

for document in full_text:
    cnt = Counter()
    for word in document:
        word = word.lower()
        if keep_word(word):
            cnt[word] += 1
    word_counts_map.append(cnt)

In [53]:
document_sentence_scores = []
for document in full_text_sentences[0]:
    sentence_scores = {}
    document_word_counts = word_counts_map[i]
    num_doc_words = sum(document_word_counts.values())
    
    for sentence_id, sentence in enumerate(document):
        sentence_word_freq_sum = 0
        num_words_in_sentence = 0
        
        for word in sentence:
            word = word.lower()
            if keep_word(word):
                word_freq = document_word_counts[word] / num_doc_words
                sentence_word_freq_sum += word_freq
                num_words_in_sentence += 1
        
        sentence_score = sentence_word_freq_sum / num_words_in_sentence if num_words_in_sentence != 0 else 0
        sentence_scores[sentence_id] = sentence_score
        
    document_sentence_scores.append(sentence_scores)

In [54]:
def get_top_sentences(doc_id, N=5):
    sentence_scores_map = document_sentence_scores[doc_id]
    
    sentence_scores = [(idx,score) for idx, score in sentence_scores_map.items()]
    sentence_scores.sort(key = lambda x: x[1], reverse=True)
    
    sentence_scores = [(full_text_sentences[doc_id][idx], score) for idx, score in sentence_scores]
    
    return sentence_scores[:N]
    
    

# Experimenting

In [56]:
# def func(document_num, sentence_num):
#     return single_number

In [57]:
def sentence_position(document_num, sentence_num):
    return 1.0 / (sentence_num ** 2)

In [74]:
from statistics import mean
import scipy.stats

def sentence_length(document_num, sentence_num, mean_sent_length=None, std_dev=5):
    if not mean_sent_length:
        mean_sent_length = mean([len(item) for item in full_text_sentences[document_num]])
    sentence = full_text_sentences[document_num][sentence_num]
        
    print(scipy.stats.norm(mean_sent_length, std_dev).pdf(len(sentence)))
    

In [76]:
import nltk
from nltk.tag import pos_tag 

def proper_noun(document_num, sentence_num):
    sentence = full_text_sentences[document_num][sentence_num]
    tagged_sent = pos_tag(sentence)
    propernouns = [word for word,pos in tagged_sent if pos == 'NNP']
    
    return len(propernouns)**2

In [77]:
proper_noun(2, 1)

LookupError: 
**********************************************************************
  Resource 'taggers/averaged_perceptron_tagger/averaged_perceptron
  _tagger.pickle' not found.  Please use the NLTK Downloader to
  obtain the resource:  >>> nltk.download()
  Searched in:
    - '/Users/mudit2103/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************